In [2]:
# Sources
# https://python-chess.readthedocs.io/en/latest/
# https://github.com/rogerfitz/tutorials/tree/master/python_chess

     -------------------------------------- 149.1/149.1 kB 3.0 MB/s eta 0:00:00


In [1]:
!pip install chess
!pip install Stockfish


                                              0.0/149.1 kB ? eta -:--:--
     -------------------------------        122.9/149.1 kB 3.6 MB/s eta 0:00:01
     -------------------------------------- 149.1/149.1 kB 2.2 MB/s eta 0:00:00


In [21]:
import chess.pgn
import pandas as pd
pd.options.display.max_columns=999
import datetime
import tqdm
import zipfile
import re

In [22]:
from stockfish import Stockfish
import chess
import chess.engine
import chess.svg
import chess.pgn
import re
import numpy as np
import pandas as pd
from statsmodels.distributions.empirical_distribution import ECDF
from scipy import stats
import matplotlib.pyplot as plt
import pickle
%matplotlib inline



In [23]:
# NUM_GAMES=2384
# rows=[]

# with open(f'data/carlsen_games.pgn') as pgn:
#     for game in tqdm.tqdm(range(NUM_GAMES)):
#         row={}
#         game = chess.pgn.read_game(pgn)
#         row['headers']=game.headers.__dict__
#         row['moves']=[x.uci() for x in game.mainline_moves()]
#         rows.append(row)
# games=pd.DataFrame(rows)
# games
# games.to_csv("data/carlsen.csv",index=False)


In [24]:
def replace_numbers(string):
    pattern = r'\d'  # Matches any digit
    result = re.sub(pattern, lambda match: 'X' * int(match.group()), string)
    return result
def get_piece_position(fen, piece):
    # Split the FEN string into its components
    fen_parts = fen.split(' ')
    
    # Extract the board position part of the FEN
    board_fen = fen_parts[0]
    
    # Remove the slashes (/) to get a string of all the pieces
    board_fen = board_fen.replace('/', '')
    board_fen = replace_numbers(board_fen)    #Find the index of the knight (N) in the string
    indices = []
    for index, char in enumerate(board_fen):
        if char == piece:
            indices.append(index)
            break
            
    # Calculate the row and column based on the index
    try:
        row = 7 - (indices[0] // 8)
        column = indices[0] % 8
        return row, column
    except:
        return -1, -1
def magnus_white(curr_game):
    if 'Black' in curr_game['headers'] and 'Carlsen, Magnus' in curr_game['headers']:
        if curr_game['headers'].index('Black') < curr_game['headers'].index('Carlsen, Magnus'):
            return False
        else:
            return True
    else:
        return True

def save_list(list_save, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(list_save, file)

def load_list(file_path):
    with open(file_path, 'rb') as file:
        chess_data = pickle.load(file)
    return chess_data
# Example usage
fen = "rnbqkbnr/pppppppN/8/8/8/8/PPNPPPPP/RBNQKBNR w KQkq - 0 1"
knight_position = get_piece_position(fen, "N")
print("Knight Position: ", knight_position)

Knight Position:  (6, 7)


In [25]:
games=pd.read_csv("data/carlsen.csv")
games['moves']=games['moves'].apply(lambda x: eval(x))
#games['headers']=games['headers'].apply(lambda x: eval(x))#to dict
games

,headers,moves
0,{'_tag_roster': {'Event': 'Bayern-chI Bank Hof...,"[d2d4, g8f6, g1f3, d7d6, b1c3, g7g6, e2e4, f8g..."
1,{'_tag_roster': {'Event': 'Bayern-chI Bank Hof...,"[d2d4, d7d5, c1g5, h7h6, g5h4, c8f5, g1f3, b8d..."
2,{'_tag_roster': {'Event': 'Bayern-chI Bank Hof...,"[e2e4, e7e5, g1f3, b8c6, f1c4, g8f6, f3g5, d7d..."
3,{'_tag_roster': {'Event': 'Nordic Cup-chT U18'...,"[c2c4, g7g6, b1c3, f8g7, d2d4, d7d6, g2g3, b8d..."
4,{'_tag_roster': {'Event': 'Nordic School-ch U1...,"[e2e4, e7e5, g1f3, g8f6, f3e5, d7d6, e5f3, f6e..."
...,...,...
2379,"{'_tag_roster': {'Event': 'Douglas IoM op', 'S...","[e2e4, b8c6, g1f3, d7d6, d2d4, g8f6, d4d5, c6b..."
2380,"{'_tag_roster': {'Event': 'Douglas IoM op', 'S...","[e2e4, e7e5, g1f3, g8f6, f3e5, d7d6, e5f3, f6e..."
2381,"{'_tag_roster': {'Event': 'Douglas IoM op', 'S...","[g1f3, b7b6, e2e4, c8b7, b1c3, e7e6, d2d4, f8b..."
2382,"{'_tag_roster': {'Event': 'Douglas IoM op', 'S...","[g1f3, d7d5, g2g3, g7g6, f1g2, f8g7, e1g1, e7e..."


In [26]:
# df[a][b][c][d] = array of CPL of piece a at location (b, c) of color d
#0 = knight, 1, a, white
#df = np.zeros((2, 8, 8,2,1,), dtype=float)
# df = [[[[[0]] * 2] * 8] * 8] * 2
# #df
# # df = np.arange(256)
# # df = np.reshape(df, (2, 8, 8,2,1))
# df[0][0][0][0]
# df[1][7][7][1]

In [27]:
engine = Stockfish("stockfish-windows-2022-x86-64-avx2.exe")
engine.set_depth(20)
engine.set_skill_level(20)
board=chess.Board()


In [28]:
pieces = ['N', 'n', 'B', 'b']

In [29]:
df = [[[[[0] for _ in range(2)] for _ in range(8)] for _ in range(8)] for _ in range(2)]


In [30]:
df[1][7][7][1]

[0]

In [ ]:
for i in tqdm.tqdm(range(0, 2000)):
    try:
        board=chess.Board()
        curr_game = games.iloc[i]
        print(curr_game['moves'][0])
        for move_number, san in enumerate(curr_game['moves']):
            move=board.push_san(san)
            engine.set_fen_position(board.fen())
            evaluation=engine.get_evaluation()
            for index, piece in enumerate(pieces):
                if(magnus_white(curr_game)):
                    if (piece == 'n' or piece == 'b'): 
                        continue
                else:
                    if (piece == 'N' or piece == 'B'):
                        continue
                row, column = get_piece_position(board.fen(), piece)
                #print(i, row, column)
                if (row == -1):
                    continue
                if (i % 10 == 0):
                    print(board.fen())
                    print(evaluation)
                df[int(index / 2)][row][column][index % 2].append(evaluation['value'])
                #print(df[int(index / 2)][row][column][index % 2])
        if (i % 100 == 0):
            save_list(df, 'data/carlsen_final_' + str(int(i/100)) + '.pkl')
    except:
        print("An exception occurred")
        continue
    

  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

d2d4
rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1
{'type': 'cp', 'value': 32}
rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1
{'type': 'cp', 'value': 32}
rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2
{'type': 'cp', 'value': 31}
rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2
{'type': 'cp', 'value': 31}
rnbqkb1r/pppppppp/5n2/8/3P4/5N2/PPP1PPPP/RNBQKB1R b KQkq - 2 2
{'type': 'cp', 'value': 32}
rnbqkb1r/pppppppp/5n2/8/3P4/5N2/PPP1PPPP/RNBQKB1R b KQkq - 2 2
{'type': 'cp', 'value': 32}
rnbqkb1r/ppp1pppp/3p1n2/8/3P4/5N2/PPP1PPPP/RNBQKB1R w KQkq - 0 3
{'type': 'cp', 'value': 60}
rnbqkb1r/ppp1pppp/3p1n2/8/3P4/5N2/PPP1PPPP/RNBQKB1R w KQkq - 0 3
{'type': 'cp', 'value': 60}
rnbqkb1r/ppp1pppp/3p1n2/8/3P4/2N2N2/PPP1PPPP/R1BQKB1R b KQkq - 1 3
{'type': 'cp', 'value': 23}
rnbqkb1r/ppp1pppp/3p1n2/8/3P4/2N2N2/PPP1PPPP/R1BQKB1R b KQkq - 1 3
{'type': 'cp', 'value': 23}
rnbqkb1r/ppp1pp1p/3p1np1/8/3P4/2N2N2/PPP1PPPP/R1BQKB1R w KQkq - 0 4
{'type': 'cp', 'v

1n3r2/rbN3bk/p6p/2pNqp2/3pp3/1B5P/PPPQ1PP1/3R1RK1 w - - 4 24
{'type': 'cp', 'value': -260}
1n3r2/rbN3bk/p6p/2pNqp2/3pp3/1B5P/PPPQ1PP1/3R1RK1 w - - 4 24
{'type': 'cp', 'value': -260}
1n3r2/rbN3bk/p6p/Q1pNqp2/3pp3/1B5P/PPP2PP1/3R1RK1 b - - 5 24
{'type': 'cp', 'value': -257}
1n3r2/rbN3bk/p6p/Q1pNqp2/3pp3/1B5P/PPP2PP1/3R1RK1 b - - 5 24
{'type': 'cp', 'value': -257}
5r2/rbNn2bk/p6p/Q1pNqp2/3pp3/1B5P/PPP2PP1/3R1RK1 w - - 6 25
{'type': 'cp', 'value': -169}
5r2/rbNn2bk/p6p/Q1pNqp2/3pp3/1B5P/PPP2PP1/3R1RK1 w - - 6 25
{'type': 'cp', 'value': -169}
5r2/rbNn2bk/p6p/Q1pNqp2/3pp3/1B4PP/PPP2P2/3R1RK1 b - - 0 25
{'type': 'cp', 'value': -302}
5r2/rbNn2bk/p6p/Q1pNqp2/3pp3/1B4PP/PPP2P2/3R1RK1 b - - 0 25
{'type': 'cp', 'value': -302}
2r5/rbNn2bk/p6p/Q1pNqp2/3pp3/1B4PP/PPP2P2/3R1RK1 w - - 1 26
{'type': 'cp', 'value': -403}
2r5/rbNn2bk/p6p/Q1pNqp2/3pp3/1B4PP/PPP2P2/3R1RK1 w - - 1 26
{'type': 'cp', 'value': -403}
2r5/rb1n2bk/p6p/QNpNqp2/3pp3/1B4PP/PPP2P2/3R1RK1 b - - 2 26
{'type': 'cp', 'value': -447}
2r5/rb

  0%|                                                                              | 1/2000 [01:27<48:19:05, 87.02s/it]

8/2r3bk/p6p/2pnq3/3pp3/6pP/PPP1QP1K/3R1R2 w - - 0 37
{'type': 'cp', 'value': -571}
8/2r3bk/p6p/2pnq3/3pp3/6pP/PPP1QP1K/3R1R2 w - - 0 37
{'type': 'cp', 'value': -571}
d2d4


...

An exception occurred
e2e4


 99%|██████████████████████████████████████████████████████████████████████████▍| 1984/2000 [38:46:24<18:43, 70.20s/it]

e2e4


 99%|██████████████████████████████████████████████████████████████████████████▍| 1985/2000 [38:48:03<19:17, 77.17s/it]

e2e4
